In [13]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, classification_report, multilabel_confusion_matrix
import tensorflow as tf
import datetime, os
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, Flatten, concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.constraints import MaxNorm
from sklearn.model_selection import GridSearchCV
from splitrepeat import splitrepeat_cv

config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8))
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

features = ['ADAS_Q1', 'ADAS_Q4', 'LDELTOTAL', 'AVDEL30MIN', 'tmab_time', 'e_memory_cg', 'e_lang_cg', 'e_divatt_cg']

categorical_features = np.in1d(features, ['MMONFLR', 'faq10', 'faq6', 'COPYSYM', 'COPYNUM', 'faq5', 'MMTREE', 'COPYHAND', 'moca_clock', 'MMSEASON', 'moca_letters', 
                                          'MMBALL', 'faq9', 'MMFLOOR', 'MMDRAW', 'MMMONTH', 'PXGENAPP', 'MMWATCH', 'CLOCKCIRC', 'faq8', 'MMHOSPIT', 'moca_naming', 
                                          'PXEXTREM', 'CLOCKNUM', 'PXMUSCUL', 'faq1', 'MMTREEDL', 'CLOCKTIME', 'PXABDOM', 'MMFLAG', 'COPYCIRC', 'MMAREA', 'faq3', 
                                          'moca_digits', 'CLOCKHAND', 'MMREAD', 'MMYEAR', 'MMREPEAT', 'moca_visuo_exec','MMHAND', 'MMBALLDL', 'PXCHEST', 'MMDATE', 
                                          'MMFLAGDL',  'MMSTATE','moca_repeat', 'MMFOLD', 'MMPENCIL', 'MMDAY',  'faq2', 'PXHEART', 'CLOCKSYM', 'faq4', 'moca_serial7', 
                                          'faq7', 'MMCITY', 'PXHEADEY', 'COPYTIME', 'PXPERIPH', 'PXSKIN', 'moca_fluency', 'moca_similarities', 'PXNECK', 'MMWRITE',
                                          'e_memory_pt', 'e_lang_pt', 'e_visspat_pt', 'e_plan_pt', 'e_organ_pt', 'e_divatt_pt','e_memory_cg', 'e_lang_cg', 
                                          'e_visspat_cg','e_plan_cg', 'e_organ_cg', 'e_divatt_cg'])


data = pd.read_csv('../data/interim/data_adni.csv')
X = data[features]
y = data['CDGLOBAL']
y.replace({0:1,1:0,2:0},inplace=True)

In [2]:
X, X_test, y, y_test = train_test_split(X, y, train_size=0.25, random_state=33433, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=33433, stratify=y)     

# RF Classifier

In [4]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_features=.5, random_state=0)

%time \
df = splitrepeat_cv(X,y,rf,splits=[111,222,333,444],repeats=[111,222,333,444,555])
df.describe()

Wall time: 3.04 s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.918056,0.868182,0.895663,0.893119,0.868182,0.918056,0.895663,0.893119,0.868182,0.918056,0.895663,0.893119
std,0.047903,0.027193,0.023025,0.020966,0.027193,0.047903,0.023025,0.020966,0.027193,0.047903,0.023025,0.020966
min,0.861111,0.829545,0.862245,0.861322,0.829545,0.861111,0.862245,0.861322,0.829545,0.861111,0.862245,0.861322
25%,0.877315,0.852273,0.878827,0.878735,0.852273,0.877315,0.878827,0.878735,0.852273,0.877315,0.878827,0.878735
50%,0.916667,0.863636,0.900510,0.898674,0.863636,0.916667,0.900510,0.898674,0.863636,0.916667,0.900510,0.898674
75%,0.958333,0.877841,0.913265,0.907092,0.877841,0.958333,0.913265,0.907092,0.877841,0.958333,0.913265,0.907092
max,0.981481,0.931818,0.933673,0.928241,0.931818,0.981481,0.933673,0.928241,0.931818,0.981481,0.933673,0.928241


## Random Search

In [5]:
from sklearn.model_selection import RandomizedSearchCV

# Search parameters
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['sqrt', 'log2', .3, .5, .7, .9]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

search_params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [6]:
rf_search_rand = RandomizedSearchCV(rf, search_params, n_iter=100,cv=3,verbose=2,random_state=33433, n_jobs=-1)
rf_search_rand.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   45.5s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.4min finished


RandomizedSearchCV(cv=3,
                   estimator=RandomForestClassifier(max_features=0.5,
                                                    random_state=555),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['sqrt', 'log2', 0.3,
                                                         0.5, 0.7, 0.9],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                        

In [7]:
rf_search_rand.best_params_

{'n_estimators': 200,
 'min_samples_split': 5,
 'min_samples_leaf': 4,
 'max_features': 0.5,
 'max_depth': 60,
 'bootstrap': True}

In [8]:
best = rf_search_rand.best_estimator_
df_rand = splitrepeat_cv(X,y,best,splits=[10,20,30],repeats=[10,20,30])
df_rand.describe()

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.923868,0.847222,0.889456,0.885545,0.847222,0.923868,0.889456,0.885545,0.847222,0.923868,0.889456,0.885545
std,0.030980,0.030656,0.010518,0.010039,0.030656,0.030980,0.010518,0.010039,0.030656,0.030980,0.010518,0.010039
min,0.879630,0.806818,0.877551,0.876263,0.806818,0.879630,0.877551,0.876263,0.806818,0.879630,0.877551,0.876263
25%,0.888889,0.806818,0.882653,0.880261,0.806818,0.888889,0.882653,0.880261,0.806818,0.888889,0.882653,0.880261
50%,0.935185,0.863636,0.887755,0.880261,0.863636,0.935185,0.887755,0.880261,0.863636,0.935185,0.887755,0.880261
75%,0.953704,0.863636,0.892857,0.890152,0.863636,0.953704,0.892857,0.890152,0.863636,0.953704,0.892857,0.890152
max,0.953704,0.875000,0.908163,0.904040,0.875000,0.953704,0.908163,0.904040,0.875000,0.953704,0.908163,0.904040


## Grid Search

In [9]:
from sklearn.model_selection import GridSearchCV

search_params = {'n_estimators': [100, 200, 300],
               'max_features': [.4, .5, .6],
               'max_depth': [50, 60, 70],
               'min_samples_split': [4, 5, 6],
               'min_samples_leaf': [3, 4, 5],
               'bootstrap': [False, True]}

rf_search_grid = GridSearchCV(rf, search_params, 
                          cv = 3, n_jobs = -1, verbose = 2)
rf_search_grid.fit(X_train, y_train)

Fitting 3 folds for each of 486 candidates, totalling 1458 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 276 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done 682 tasks      | elapsed:   32.7s
[Parallel(n_jobs=-1)]: Done 1104 tasks      | elapsed:   57.6s
[Parallel(n_jobs=-1)]: Done 1458 out of 1458 | elapsed:  1.3min finished


GridSearchCV(cv=3,
             estimator=RandomForestClassifier(max_features=0.5,
                                              random_state=555),
             n_jobs=-1,
             param_grid={'bootstrap': [False, True], 'max_depth': [50, 60, 70],
                         'max_features': [0.4, 0.5, 0.6],
                         'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [4, 5, 6],
                         'n_estimators': [100, 200, 300]},
             verbose=2)

In [10]:
rf_search_grid.best_params_

{'bootstrap': True,
 'max_depth': 50,
 'max_features': 0.5,
 'min_samples_leaf': 5,
 'min_samples_split': 4,
 'n_estimators': 100}

In [11]:
best = rf_search_grid.best_estimator_
df_grid = splitrepeat_cv(X,y,best,splits=[10,20,30],repeats=[10,20,30])
df_grid.describe()

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.916667,0.844697,0.884354,0.880682,0.844697,0.916667,0.884354,0.880682,0.844697,0.916667,0.884354,0.880682
std,0.028161,0.032640,0.011120,0.011367,0.032640,0.028161,0.011120,0.011367,0.032640,0.028161,0.011120,0.011367
min,0.870370,0.795455,0.872449,0.865320,0.795455,0.870370,0.872449,0.865320,0.795455,0.870370,0.872449,0.865320
25%,0.888889,0.818182,0.877551,0.872685,0.818182,0.888889,0.877551,0.872685,0.818182,0.888889,0.877551,0.872685
50%,0.925926,0.863636,0.877551,0.876263,0.863636,0.925926,0.877551,0.876263,0.863636,0.925926,0.877551,0.876263
75%,0.935185,0.863636,0.892857,0.889099,0.863636,0.935185,0.892857,0.889099,0.863636,0.935185,0.892857,0.889099
max,0.953704,0.875000,0.903061,0.900463,0.875000,0.953704,0.903061,0.900463,0.875000,0.953704,0.903061,0.900463


In [12]:
rf = RandomForestClassifier(n_estimators=100, max_features=.5, max_depth = 50, min_samples_split = 4, min_samples_leaf = 5, bootstrap=True, random_state=33433)

%time \
df = splitrepeat_cv(X,y,rf,splits=[111,222,333,444],repeats=[111,222,333,444,555])
df.describe()

Wall time: 2.99 s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.932870,0.872727,0.905867,0.902799,0.872727,0.932870,0.905867,0.902799,0.872727,0.932870,0.905867,0.902799
std,0.042044,0.044027,0.025776,0.025704,0.044027,0.042044,0.025776,0.025704,0.044027,0.042044,0.025776,0.025704
min,0.870370,0.818182,0.862245,0.860269,0.818182,0.870370,0.862245,0.860269,0.818182,0.870370,0.862245,0.860269
25%,0.898148,0.840909,0.885204,0.880314,0.840909,0.898148,0.885204,0.880314,0.840909,0.898148,0.885204,0.880314
50%,0.925926,0.863636,0.908163,0.906145,0.863636,0.925926,0.908163,0.906145,0.863636,0.925926,0.908163,0.906145
75%,0.967593,0.897727,0.924745,0.925295,0.897727,0.967593,0.924745,0.925295,0.897727,0.967593,0.924745,0.925295
max,0.990741,0.943182,0.943878,0.938552,0.943182,0.990741,0.943878,0.938552,0.943182,0.990741,0.943878,0.938552


In [14]:
# All samples

%time \
df = splitrepeat_cv(X,y,rf,splits=list(int(x)*42+42 for x in range(10)),repeats=list(int(x)*42+42 for x in range(10)), initial_split_seed=33433, initial_split_ratio=.25)
df.describe()

Wall time: 17.2 s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.917349,0.840508,0.882653,0.878929,0.840508,0.917349,0.882653,0.878929,0.840508,0.917349,0.882653,0.878929
std,0.018767,0.031337,0.016056,0.016874,0.031337,0.018767,0.016056,0.016874,0.031337,0.018767,0.016056,0.016874
min,0.883721,0.779661,0.862245,0.854947,0.779661,0.883721,0.862245,0.854947,0.779661,0.883721,0.862245,0.854947
25%,0.905814,0.813559,0.867347,0.865083,0.813559,0.905814,0.867347,0.865083,0.813559,0.905814,0.867347,0.865083
50%,0.911628,0.830508,0.882653,0.877546,0.830508,0.911628,0.882653,0.877546,0.830508,0.911628,0.882653,0.877546
75%,0.931395,0.864407,0.890306,0.887643,0.864407,0.931395,0.890306,0.887643,0.864407,0.931395,0.890306,0.887643
max,0.948837,0.898305,0.923469,0.921246,0.898305,0.948837,0.923469,0.921246,0.898305,0.948837,0.923469,0.921246


# Outputs

In [15]:
### Save outputs ###
q = 'CDR0_Boruta8_2'

df['set'] = q
df.to_csv(('../models/outputs/' + q + '.csv'), index=False)